<a href="https://colab.research.google.com/github/Svar7769/periodic_review_inventory_system/blob/master/RL_inv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [307]:
!pip install tensorboard stable-baselines3

In [18]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Parameters
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)
lead_time = 2  # Lead time for order arrival
initial_inventory = 100
max_order = 50
review_period = 7

# Generate dates
dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Create list to store data
data_list = []

# Initialize inventory and order arrival time
inventory_level = initial_inventory
order_arrival_time = 0
order_quantity = 0  # Define order_quantity outside of the loop

for date in dates:
    # Simulate demand
    demand = np.random.randint(0, 20)  # Random demand between 0 and 19

    # Place order if it's a review period
    if date.day % review_period == 0:
        order_quantity = np.random.randint(0, max_order + 1)  # Random order quantity between 0 and max_order
        order_arrival_time = lead_time

    # Update inventory level based on demand and order arrival
    inventory_level = max(inventory_level + order_arrival_time - demand, 0)

    # Update order arrival time
    if order_arrival_time > 0:
        order_arrival_time -= 1

    # Append data to list
    data_list.append({
        'Date': date,
        'Product_ID': 1,  # Assuming Product_ID is 1 for simplicity
        'Demand': demand,
        'Order_Quantity': order_quantity if order_arrival_time == 0 else 0,
        'Order_Arrival_Time': order_arrival_time
    })

# Convert list of dictionaries to DataFrame
data = pd.DataFrame(data_list)

# Convert 'Order_Quantity' and 'Order_Arrival_Time' to integers
data['Order_Quantity'] = data['Order_Quantity'].astype(int)
data['Order_Arrival_Time'] = data['Order_Arrival_Time'].astype(int)

# Display the first few rows of the simulated dataset
print(data.head())


        Date  Product_ID  Demand  Order_Quantity  Order_Arrival_Time
0 2023-01-01           1      15               0                   0
1 2023-01-02           1      18               0                   0
2 2023-01-03           1      13               0                   0
3 2023-01-04           1       5               0                   0
4 2023-01-05           1       3               0                   0


In [308]:
import numpy as np
import pandas as pd
from stable_baselines3 import DQN
from itertools import product
import random
import matplotlib.pyplot as plt
import gymnasium as gym  # Updated import to use gymnasium instead of gym
from gymnasium import spaces  # Import spaces from gymnasium
import os  # For operating system-related functions
from stable_baselines3.common.env_checker import check_env  # For checking the custom environment
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import configure


In [ ]:
class TensorboardCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(TensorboardCallback, self).__init__(verbose)

    def _on_step(self) -> bool:
        # Log scalar metrics
        self.logger.record('reward', self.locals['rewards'])
        self.logger.record('episode_length', self.locals['infos'][0]['episode']['l'])
        return True


# New Section

In [302]:
def barGraph(row):
    date = row['Date'].values[0]
    product_id = row['Product_ID'].values[0]
    demand = row['Demand'].values[0]
    order_quantity = row['Order_Quantity'].values[0]
    order_arrival_time = row['Order_Arrival_Time'].values[0]

    # Create a bar plot
    plt.figure(figsize=(10, 6))
    plt.bar(['Demand', 'Order Quantity', 'Order Arrival Time'], [demand, order_quantity, order_arrival_time])
    plt.xlabel('Metrics')
    plt.ylabel('Quantity')
    plt.title(f'Data for Product {product_id} on {date}')
    plt.show()

class InventoryEnv(gym.Env):
    def __init__(self, data):
        super(InventoryEnv, self).__init__()
        self.data = data
        self.product_id = 1  # Assuming Product_ID is 1 for the single-product environment
        self.num_days = data['Date'].nunique()

        # Inventory management parameters
        self.initial_inventory = 50
        self.max_order = 50
        self.holding_cost = 1
        self.shortage_cost = 5
        self.order_cost = 2
        self.review_period = 7
        self.lead_time = 2
        self.daily_profit = 0
        self.cumulative_profit = 0
        self.threshold_inventory = 100
        self.threshold_penalty_cost = 100

        # Define action and observation space
        self.action_space = spaces.Discrete(self.max_order + 1)  # Orders can be from 0 to max_order
        self.observation_space = spaces.Box(low=0, high=self.initial_inventory, shape=(1,), dtype=np.int32)

        self.reset()

    def reset(self, seed=None, options=None):
        self.current_day = 0
        self.inventory_level = self.initial_inventory
        self.total_cost = 0
        self.cumulative_profit = 0
        self.daily_profit = 0
        self.reward = 0
        return np.array([self.inventory_level], dtype=np.int32), info

    def step(self, action):
        truncated = False

        order_quantity = int(np.ceil(action / 12)) * 12 if action > 12 else 0
        # Convert current_day to date
        current_date = self.data['Date'].min() + timedelta(days=self.current_day)
        # Calculate arrival date for the order
        arrival_date = current_date + timedelta(days=self.lead_time)

        #print(f'Order Quantity : {order_quantity}')

        self.data.loc[(self.data['Product_ID'] == self.product_id) & (
            self.data['Date'] == current_date), 'Order_Quantity'] = order_quantity

        #print(f'OrderQuantity : {order_quantity}, arrival date : {arrival_date}')

        # Update 'Order_Arrival_Time' for the corresponding product and arrival date
        if arrival_date <= self.data['Date'].max():
            self.data.loc[(self.data['Product_ID'] == self.product_id) & (
                self.data['Date'] == arrival_date), 'Order_Arrival_Time'] =+ order_quantity

        # Get demand for the current date and product
        demand_today_row = self.data.loc[(self.data['Product_ID'] == self.product_id) & (
            self.data['Date'] == current_date), 'Demand']
        demand_today = demand_today_row.values[0] if not demand_today_row.empty else 0

        #print(f'Demand Today : {demand_today}')

        # Calculating selling cost, buying cost and profit
        selling_price = 2.5
        buying_price = 2.0

        order_today = self.data.loc[(self.data['Product_ID'] == self.product_id) & (
            self.data['Date'] == current_date), 'Order_Arrival_Time'].sum()

        # Apply discount factor
        discount_factor = 0.9 if order_today > 24 else 1.0

        # Adjust inventory levels based on demand
        self.inventory_level = self.inventory_level + order_today - demand_today


        # Calculate and update costs for the day
        holding_costs = self.holding_cost * max(self.inventory_level, 0)  # Only apply holding costs for positive inventory
        ordering_costs = buying_price * order_today * discount_factor
        selling_cost = demand_today * selling_price

        # Calculate profit
        profit = selling_cost - ordering_costs

        # Add positive cost when profits are high
        positive_cost = 1.5

        # Calculate penalties
        threshold_penalty = self.threshold_penalty_cost * max(self.inventory_level - self.threshold_inventory, 0)
        shortage_penalty = self.shortage_cost * max(-self.inventory_level, 0)  # Penalty for negative inventory

        # Update daily and cumulative profits
        self.daily_profit = profit
        self.cumulative_profit += self.daily_profit

        ## Calculate total reward
        self.reward = profit - holding_costs - threshold_penalty - shortage_penalty

        # Update total cost
        self.total_cost += holding_costs + ordering_costs + threshold_penalty + shortage_penalty

        reward = self.reward

        # Move to the next date
        self.current_day += 1
        done = self.current_day >= self.num_days
        return np.array([self.inventory_level], dtype=np.int32), float(reward), done, truncated,{}

    def plot_inventory_level(self):
        # Calculate inventory levels for each day
        inventory_levels = []
        for day in range(self.num_days):
            inventory_level = self.initial_inventory
            for i in range(day):
                current_date = self.data['Date'].min() + timedelta(days=i)
                demand_today_row = self.data.loc[(self.data['Product_ID'] == self.product_id) & (
                    self.data['Date'] == current_date), 'Demand']
                demand_today = demand_today_row.values[0] if not demand_today_row.empty else 0
                order_today = self.data.loc[(self.data['Product_ID'] == self.product_id) & (
                    self.data['Date'] == current_date), 'Order_Arrival_Time'].sum()
                inventory_level = inventory_level + order_today - demand_today
            inventory_levels.append(inventory_level)

        # Create a line plot
        plt.figure(figsize=(12, 6))
        plt.plot(range(self.num_days), inventory_levels, marker='o')
        plt.xlabel('Day')
        plt.ylabel('Inventory Level')
        plt.title('Inventory Level Over Time')
        plt.grid(True)
        plt.show()


    def render(self, mode='human'):
        current_date = self.data['Date'].min() + timedelta(days=self.current_day)
        row = self.data[(self.data['Product_ID'] == self.product_id) & (self.data['Date'] == current_date)]
        OrderToday = row['Order_Arrival_Time'].sum()
        print(
              f'Day: {self.current_day}, Inventory Level: {self.inventory_level}, Reward: {self.reward}, '
              f'Daily Profit: {self.daily_profit}, Cumulative Profit: {self.cumulative_profit}, OrderToday: {OrderToday}')
        #print(row)





In [303]:
# Check if the necessary columns are present
required_columns = ['Date', 'Product_ID', 'Demand', 'Order_Quantity', 'Order_Arrival_Time']
missing_columns = [col for col in required_columns if col not in data.columns]

if missing_columns:
    raise ValueError(f"Missing columns in the dataset: {missing_columns}")

# Initialize the environment with the synthetic data
env = InventoryEnv(data)

# Verify Observation Space and Action Space
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

# Run the environment for a few episodes
episodes = 1
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0
    while not done:
        action = env.action_space.sample()
        # print(f'action : {action}')
        next_state, reward, done,truc, info = env.step(action)
        score += reward
        # print(f'Next state : {next_state}, Reward : {reward}, done : {done}, info: {info}')
        # print(f"Action taken: {action}, Orders: {env.actions[action]}")
        env.render()
    print(f"Episode: {episode} ended with Score: {score}")
    #env.plot_inventory_level()



Observation space: Box(0, 50, (1,), int32)
Action space: Discrete(51)
Day: 1, Inventory Level: 35, Reward: 2.5, Daily Profit: 37.5, Cumulative Profit: 37.5, OrderToday: 0
Day: 2, Inventory Level: 17, Reward: 28.0, Daily Profit: 45.0, Cumulative Profit: 82.5, OrderToday: 24
Day: 3, Inventory Level: 28, Reward: -43.5, Daily Profit: -15.5, Cumulative Profit: 67.0, OrderToday: 0
Day: 4, Inventory Level: 23, Reward: -10.5, Daily Profit: 12.5, Cumulative Profit: 79.5, OrderToday: 24
Day: 5, Inventory Level: 44, Reward: -84.5, Daily Profit: -40.5, Cumulative Profit: 39.0, OrderToday: 24
Day: 6, Inventory Level: 66, Reward: -109.0, Daily Profit: -43.0, Cumulative Profit: -4.0, OrderToday: 0
Day: 7, Inventory Level: 52, Reward: -17.0, Daily Profit: 35.0, Cumulative Profit: 31.0, OrderToday: 24
Day: 8, Inventory Level: 61, Reward: -71.5, Daily Profit: -10.5, Cumulative Profit: 20.5, OrderToday: 0
Day: 9, Inventory Level: 49, Reward: -19.0, Daily Profit: 30.0, Cumulative Profit: 50.5, OrderToday:

In [258]:
env = InventoryEnv(data)

# Assuming your environment is called 'env'
check_env(env)


In [305]:
# Check if the necessary columns are present
#required_columns = ['Date', 'Product_ID', 'Demand', 'Order_Quantity', 'Order_Arrival_Time', 'Inventory_Level', 'Holding_Cost', 'Shortage_Cost', 'Ordering_Cost', 'Total_Cost']
required_columns = ['Date', 'Product_ID', 'Demand', 'Order_Quantity', 'Order_Arrival_Time']
missing_columns = [col for col in required_columns if col not in data.columns]

print(f'data Coloums : {data.columns}')



if missing_columns:
    raise ValueError(f"Missing columns in the dataset: {missing_columns}")

# Initialize the environment with the synthetic data
env = InventoryEnv(data)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_log_dir = log_dir + "/tensorboard/"

# Configure the logger
new_logger = configure(log_dir, ["stdout", "csv", "tensorboard"])


# Train the RL model
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=tensorboard_log_dir)
model.set_logger(new_logger)
callback = TensorboardCallback()
model.learn(total_timesteps=10000, callback = callback)

# Save the trained model
model.save("inventory_management_model")



data Coloums : Index(['Date', 'Product_ID', 'Demand', 'Order_Quantity', 'Order_Arrival_Time'], dtype='object')
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 365       |
|    ep_rew_mean      | -1.41e+08 |
|    exploration_rate | 0.05      |
| time/               |           |
|    episodes         | 4         |
|    fps              | 184       |
|    time_elapsed     | 7         |
|    total_timesteps  | 1460      |
| train/              |           |
|    learning_rate    | 0.0001    |
|    loss             | 4.14e+05  |
|    n_updates        | 339       |
-----------------------------------
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 365       |
|    ep_rew_mean      | -1.47e+08 |
|    exploration_rate | 0.05      |
| time/               |           |
|    episodes         | 8         |
|  

In [306]:
# Verify Observation Space and Action Space
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)


# Test the RL model
episodes = 1
for episode in range(1, episodes+1):
    state, info = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = model.predict(state)[0]
        next_state, reward, done,truc, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
    #env.plot_inventory_level()

Observation space: Box(0, 50, (1,), int32)
Action space: Discrete(51)
Day: 0, Inventory Level: 50, Reward: 0, Daily Profit: 0, Cumulative Profit: 0, OrderToday: 0
Day: 1, Inventory Level: 35, Reward: 2.5, Daily Profit: 37.5, Cumulative Profit: 37.5, OrderToday: 0
Day: 2, Inventory Level: 17, Reward: 28.0, Daily Profit: 45.0, Cumulative Profit: 82.5, OrderToday: 0
Day: 3, Inventory Level: 4, Reward: 28.5, Daily Profit: 32.5, Cumulative Profit: 115.0, OrderToday: 0
Day: 4, Inventory Level: -1, Reward: 7.5, Daily Profit: 12.5, Cumulative Profit: 127.5, OrderToday: 0
Day: 5, Inventory Level: -4, Reward: -12.5, Daily Profit: 7.5, Cumulative Profit: 135.0, OrderToday: 0
Day: 6, Inventory Level: -6, Reward: -25.0, Daily Profit: 5.0, Cumulative Profit: 140.0, OrderToday: 0
Day: 7, Inventory Level: -20, Reward: -65.0, Daily Profit: 35.0, Cumulative Profit: 175.0, OrderToday: 0
Day: 8, Inventory Level: -35, Reward: -137.5, Daily Profit: 37.5, Cumulative Profit: 212.5, OrderToday: 0
Day: 9, Inven